In [1]:
import keras  
from gensim.models import Word2Vec

Using TensorFlow backend.
D:\ANACONDA\envs\tensorflownt\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [34]:
import numpy as np   
from keras.utils import plot_model,np_utils  
from keras.preprocessing.image import ImageDataGenerator  
from keras.models import *  
from keras.layers import *  
from keras.callbacks import *  
from keras import backend as K  
import h5py 

# Load Data

In [35]:
filename = "cp.txt"  
raw_text = open(filename).read()  
raw_text = raw_text.lower()

In [36]:
chars = sorted(list(set(raw_text)))  
char_to_int = dict((c, i) for i, c in enumerate(chars))  
int_to_char = dict((i, c) for i, c in enumerate(chars)) 

In [77]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '–', '‘', '’']


In [75]:
n_chars = len(raw_text)  
n_vocab = len(chars)

In [39]:
print('vocal: ',n_vocab)

vocal:  40


In [40]:
seq_length = 16  #how to decide
dataX = []  
dataY = [] 

In [41]:
for i in range(0, n_chars - seq_length, 1):  
    seq_in = raw_text[i:i + seq_length]  
    seq_out = raw_text[i + seq_length]  
    dataX.append([char_to_int[char] for char in seq_in])  
    dataY.append(char_to_int[seq_out])

In [42]:
print(dataX[:3])
print(dataY[:3])

[[23, 26, 26, 22, 1, 12, 30, 1, 30, 19, 16, 1, 29, 30, 12, 28], [26, 26, 22, 1, 12, 30, 1, 30, 19, 16, 1, 29, 30, 12, 28, 29], [26, 22, 1, 12, 30, 1, 30, 19, 16, 1, 29, 30, 12, 28, 29, 0]]
[29, 0, 23]


In [43]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns) 

Total Patterns:  19159


In [44]:
# reshape X to be [samples, time steps, features]  
X = np.reshape(dataX, (n_patterns, seq_length, 1))  
#normalize
X = X / float(n_vocab)

In [45]:
Y = np_utils.to_categorical(dataY)  

In [46]:
#Set heckpoints and save weights
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"  
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')  
#callbacks_list = [checkpoint]  

# Model

In [51]:
K.clear_session()

In [52]:
model = Sequential()  
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True, name='LSTM1'))  
model.add(Dropout(0.2, name='Droupout1'))
model.add(LSTM(128,input_shape=(X.shape[1], X.shape[2]),return_sequences=True,name='LSTM2')) 
model.add(Dropout(0.2,name='Droupout2'))
model.add(Flatten(name='Flatten'))  
model.add(Dense(n_vocab,activation='softmax',name='output'))  

adam=keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy']) 

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 16, 128)           66560     
_________________________________________________________________
Droupout1 (Dropout)          (None, 16, 128)           0         
_________________________________________________________________
LSTM2 (LSTM)                 (None, 16, 128)           131584    
_________________________________________________________________
Droupout2 (Dropout)          (None, 16, 128)           0         
_________________________________________________________________
Flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
output (Dense)               (None, 40)                81960     
Total params: 280,104
Trainable params: 280,104
Non-trainable params: 0
_________________________________________________________________


In [53]:
#import pydot
#import graphviz
#plot_model(model, to_file='model.png') 

# Training the Model

In [54]:
nit = 20   # number of training iterations
nepoch_per_it = 20  # number of epochs per iterations

# Loss, accuracy and epoch per iteration
loss = np.zeros(nit)
acc = np.zeros(nit)
epoch_it = np.zeros(nit)

# Main iteration loop
for it in range(nit):
    
    # Continue the fit of the model
    init_epoch = it*nepoch_per_it
    model.fit(X, Y, epochs=nepoch_per_it, batch_size=32,verbose=0)#, callbacks=callbacks_list)  
    
    # Measure the loss and accuracy on the training data
    lossi, acci = model.evaluate(X,Y)
    epochi = (it+1)*nepoch_per_it
    epoch_it[it] = epochi
    loss[it] = lossi
    acc[it] = acci
    print("epoch=%4d loss=%12.4e acc=%7.5f" % (epochi,lossi,acci))
model.save('lyrics_pre.h5')  

19159/19159 [==============================] - 7s 372us/step
epoch=  20 loss=  8.5355e-01 acc=0.76366
19159/19159 [==============================] - 7s 383us/step
epoch=  40 loss=  2.7769e-01 acc=0.92474
19159/19159 [==============================] - 7s 372us/step
epoch=  60 loss=  1.0570e-01 acc=0.97260
19159/19159 [==============================] - 7s 377us/step
epoch=  80 loss=  8.5079e-02 acc=0.97495
19159/19159 [==============================] - 7s 365us/step
epoch= 100 loss=  7.0698e-02 acc=0.97797
19159/19159 [==============================] - 7s 359us/step
epoch= 120 loss=  6.7028e-02 acc=0.97823
19159/19159 [==============================] - 7s 363us/step
epoch= 140 loss=  6.1115e-02 acc=0.98058
19159/19159 [==============================] - 7s 342us/step
epoch= 160 loss=  6.0273e-02 acc=0.97928
19159/19159 [==============================] - 7s 343us/step
epoch= 180 loss=  5.6096e-02 acc=0.98084
19159/19159 [==============================] - 7s 358us/step
epoch= 200 loss=  5.7

# Generate Lyrics

In [125]:
def predict_next(input_array):
    x = np.reshape(input_array,(1,seq_length,1))
    x = x/float(n_vocab)
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    res = []
    if len(raw_input)<seq_length:
        for i in range(seq_length-len(raw_input)):
            res.append(1)
    for c in raw_input[-seq_length:]:
        res.append(char_to_int[c])
    return res

def y_to_char(y):
    #n = 2
    #largest = y.argsort()[0]
    #largest = largest[-2:]
    #top_n = np.zeros(n)
    #for i in range(n):
    #    top_n[i]=largest[i]
    #np.random.shuffle(top_n)
    #c = int_to_char[top_n[0]]
    largest = y.argmax()
    c = int_to_char[largest]
    return c

In [126]:
def generate_article(init, rounds=500):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string +=n
    return in_string

In [127]:
init = "we are diamonds taking shape"
article = generate_article(init)
print(article)

we are diamonds taking shape

if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure oh then i
and if we’ve only got this life
this adventure
